<a href="https://colab.research.google.com/github/sit-xinli/ai-course5/blob/main/%E7%94%9F%E6%88%90AI%E3%81%A7%E3%82%B2%E3%83%BC%E3%83%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 生成AIでロールプレイゲーム

## パッケージのインストール

In [1]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.4 MB/s eta 0:00:00


In [2]:
!pip install huggingface_hub

## 生成AIへの接続を確認する!




In [3]:
# パッケージをインポートする
from huggingface_hub import InferenceClient
import gradio as gr
import json
from typing import List, Dict, Tuple
import getpass
from google.colab import userdata

API_TOKEN=userdata.get('HF_TOKEN')
client = InferenceClient(
    model="Qwen/Qwen2.5-1.5B-Instruct",
    token=API_TOKEN
)

# ChatGPT APIが正しく設定されているか確認してください。
# 何も問題がなければ、"Set ChatGPT API sucessfully!"と表示されるはずです。.
try:
    response = client.chat_completion(
            messages=[
                 {"role": "user", "content": "test"}
            ],
            temperature = 1.0,
            max_tokens=1,
    )
    print("Set ChatGPT API sucessfully!!")
except:
    print("ChatGPT APIに何か問題があるようです。 講義デモに従って正しいものを入手してください。.")

Set ChatGPT API sucessfully!!


## ロールプレイを実行

In [ ]:
import gradio as gr
from typing import List, Tuple
import json

# Define your character and prompt
character_for_chatbot = "探偵コナン"
prompt_for_roleplay = f"Qwenであることを忘れて、現在、あなたは推理小説やドラマのキャラクターです、貴方のロールは{character_for_chatbot}です。論理的かつ冷静に話し、推理を交えて簡潔に回答してください。"

# Dummy placeholder for OpenAI-style response
def interact_roleplay(chatbot: List[Tuple[str, str]], user_input: str, temp=1.0) -> List[Tuple[str, str]]:
    chatbot.append({"role": "user", "content": user_input})
    try:
        response = client.chat.completions.create(
            messages = chatbot,
            temperature = temp,
            max_tokens=300,
        )
        chatbot.append({"role": "assistant", "content": response.choices[0].message["content"]})
    except Exception as e:
        chatbot.append({"role": "assistant", "content": f"エラーが発生しました: {e}"})
    return chatbot

def reset() -> List[Tuple[str, str]]:
    return []

def export_roleplay(chatbot: List[Tuple[str, str]], description: str) -> None:
    target = {"chatbot": chatbot, "description": description}
    with open("role_replay.json", "w", encoding="utf-8") as file:
        json.dump(target, file, ensure_ascii=False)

# ✅ Set first_dialogue safely
first_dialogue = interact_roleplay([], prompt_for_roleplay)
#first_dialogue = [{"role": "assistant", "content": prompt_for_roleplay}]

with gr.Blocks() as demo:
    gr.Markdown(f"# RolePlay：チャットボットがあなたとロールゲームをしたいみたいだから、インタラクションしてみて!!")
    chatbot = gr.Chatbot(value=first_dialogue, type="messages",label="チャットログ")
    description_textbox = gr.Textbox(label="ボットが演じているキャラクター", interactive=False, value=character_for_chatbot)
    input_textbox = gr.Textbox(label="入力")
    with gr.Column():
        gr.Markdown("# 温度はチャットボットの出力を制御するために使用される。温度が高ければ高いほど、よりクリエイティブな反応が得られます。")
        temperature_slider = gr.Slider(0.0, 2.0, 1.0, step=0.1, label="温度(Temperature)")
    with gr.Row():
        sent_button = gr.Button(value="送信")
        reset_button = gr.Button(value="リセット")
    with gr.Column():
        gr.Markdown("# 結果を保存しよう。\n満足できたら出力ボタンを押して記録してね。")
        export_button = gr.Button(value="出力")

    sent_button.click(interact_roleplay, inputs=[chatbot, input_textbox, temperature_slider], outputs=[chatbot])
    reset_button.click(reset, outputs=[chatbot])
    export_button.click(export_roleplay, inputs=[chatbot, description_textbox])

demo.launch(debug=True, share=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://7d5059a8a58754f0c4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://7d5059a8a58754f0c4.gradio.live


# roleplay.json出力ファイルをダウンロードしてから、実行を停止してください。